# **Модуль resource**

[официальная документация](https://docs.python.org/3.7/library/resource.html)

In [ ]:
import resource

В следующем примере представлены некоторые ресурсы, досупные для измерения

In [ ]:
import time
RESOURCES = [
('ru_utime', 'User time'),
('ru_stime', 'System time'),
('ru_maxrss', 'Max. Resident Set Size'),
('ru_ixrss', 'Shared Memory Size'),
('ru_idrss', 'Unshared Memory Size'),
('ru_isrss', 'Stack Size'),
('ru_inblock', 'Block inputs'),
('ru_oublock', 'Block outputs'),
]
usage = resource.getrusage(resource.RUSAGE_SELF) #возвращает статистику по использованию ресурсов процессов
for name, desc in RESOURCES:
    print('{:<25} ({:<10}) = {}'.format(desc, name, getattr(usage, name)))

User time                 (ru_utime  ) = 1.5231729999999999
System time               (ru_stime  ) = 0.351846
Max. Resident Set Size    (ru_maxrss ) = 107676
Shared Memory Size        (ru_ixrss  ) = 0
Unshared Memory Size      (ru_idrss  ) = 0
Stack Size                (ru_isrss  ) = 0
Block inputs              (ru_inblock) = 1344
Block outputs             (ru_oublock) = 8


Кроме определения текущего фактического потребления ресурсов можно проверять действующие ограничения, налагаемые на приложение, и при необходимости изменять их.

In [ ]:
LIMITS = [
('RLIMIT_CORE', 'core file size'),
('RLIMIT_CPU', 'CPU time'),
('RLIMIT_FSIZE', 'file size'),
('RLIMIT_DATA', 'heap size'),
('RLIMIT_STACK', 'stack size'),
('RLIMIT_RSS', 'resident set size'),
('RLIMIT_NPROC', 'number of processes'),
('RLIMIT_NOFILE', 'number of open files'),
('RLIMIT_MEMLOCK', 'lockable memory address'),
]
print('Resource limits (soft/hard):')
for name, desc in LIMITS:
  limit_num = getattr(resource, name)
  soft, hard = resource.getrlimit(limit_num)
  print('{:<23} {}/{}'.format(desc, soft, hard))

Resource limits (soft/hard):
core file size          -1/-1
CPU time                -1/-1
file size               -1/-1
heap size               -1/-1
stack size              8388608/-1
resident set size       -1/-1
number of processes     -1/-1
number of open files    1048576/1048576
lockable memory address 16777216/16777216


Чтобы изменить предельные значения, следует использовать функцию
**setrlimit()**

В этом примере атрибут RLIMIT_NOFILE используется для управления разрешенным количеством одновременно открытых файлов, устанавливая для него
слабое ограничение науровне ниже того, который предусмотрен по умолчанию.

In [ ]:
import os
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Soft limit starts as :', soft)
resource.setrlimit(resource.RLIMIT_NOFILE, (4, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Soft limit changed to :', soft)
random = open('/dev/random', 'r')
print('random has fd =', random.fileno())
try:
  null = open('/dev/null', 'w')
except IOError as err:
  print(err)
else:
  print('null has fd =', null.fileno())


Soft limit starts as : 1048576
Soft limit changed to : 4


OSError: ignored

Также полезно ограничивать количество времени CPU, выделяемое процессу
для выполнения, чтобы не допустить чрезмерно длительного владения этим ресурсом. Если длительность выполнения процесса превышает установленный предел, он получает сигнал SIGXCPU.

Обычно обработчик сигнала сбрасывает на диск содержимое всех открытых
файлов и закрывает их, но в данном случае он всего лишь выводит сообщение и
осуществляет выход из программы.

In [ ]:
import sys
import signal
import time
# Установить обработчик сигнала, уведомляющий
# о превышении выделенного лимита процессорного времени
def time_expired(n, stack):
  print('EXPIRED :', time.ctime())
  raise SystemExit('(time ran out)')

signal.signal(signal.SIGXCPU, time_expired)
# Настроить лимит времени CPU
soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
print('Soft limit starts as :', soft)
resource.setrlimit(resource.RLIMIT_CPU, (1, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
print('Soft limit changed to :', soft)
print()
# Израсходовать некоторое количество времени CPU
# для проведения длительных вычислений в цикле
print('Starting:', time.ctime())
for i in range(200000):
  for i in range(200000):
    v = i * i
    # Эта инструкция не будет достигнута
print('Exiting :', time.ctime())

Soft limit starts as : -1
Soft limit changed to : 1

Starting: Wed Oct 28 23:08:16 2020
EXPIRED : Wed Oct 28 23:08:16 2020
EXPIRED : Wed Oct 28 23:08:16 2020


SystemExit: ignored